In [19]:
import codecs
from time import perf_counter
import numpy as np

## Наивный алгоритм

In [20]:
def naive_matcher(text, pattern):
    operations = 0
    n = len(text)
    m = len(pattern)
    results = []
    for i in range(n - m + 1):
        j = 0
        while j < m:
            operations += 1
            if text[i + j] != pattern[j]:
                break
            j += 1
        if j == m:
            results.append((i, i + j - 1))
    return results, operations

## Алгоритм Рабина-Карпа

In [33]:
def RK_matcher(text, pattern, q=117, d = 256):
    operations = 0
    m = len(pattern)
    n = len(text)
    pattHash = 0
    txtHash = 0
    results = []
    h = pow(d, m-1) % q
  
    for i in range(m):
        pattHash = (d*pattHash + ord(pattern[i]))%q
        txtHash = (d*txtHash + ord(text[i]))%q
  
    for i in range(n-m+1):
        operations += 1 # hash comp
        if pattHash==txtHash:
            
            for j in range(m):
                operations += 1 # character comp
                if text[i+j] != pattern[j]:
                    break
                else: 
                    j+=1
            if j==m:
                results.append((i, i + m - 1))
                
        if i < n-m:
            txtHash = (d*(txtHash-ord(text[i])*h) + ord(text[i+m]))%q
            if txtHash < 0:
                txtHash = txtHash + q
    return results, operations

## Алгоритм Бойера-Мура-Хорспула

In [39]:
def BMH_matcher(text, pattern):
    operations = 0
    results = []
    m = len(pattern)
    n = len(text)
    if m > n:
        return
    skip = []
    for k in range(2048): 
        skip.append(m)
    for k in range(m - 1): 
        skip[ord(pattern[k])] = m - k - 1
    skip = tuple(skip)
    k = m - 1
    while k < n:
        j = m - 1
        i = k
        while j >= 0 and text[i] == pattern[j]:
            operations += 1 # char comp
            j -= 1
            i -= 1
        operations += 1 # index comp
        if j == -1: 
            results.append((i + 1, i + m))
        k += skip[ord(text[k])]
    return results, operations

## Алгоритм Кнутта-Мориса-Пратта

In [44]:
def compute_prefix(pattern):
    m = len(pattern)
    pref_fun = [0] * len(pattern)
    k = 0
    for i in range(1, m):
        while k > 0 and pattern[k] != pattern[i]:
            k = pref_fun[k - 1]
        if pattern[k] == pattern[i]:
            k += 1
        pref_fun[i] = k
    return pref_fun

def KMP_matcher(text, pattern):
    operations = 0
    n = len(text)
    m = len(pattern)
    results = []
    prefix_fun = compute_prefix(pattern)
    q = 0

    for i in range(n):
        while q > 0 and pattern[q] != text[i]:
            operations += 1
            q = prefix_fun[q - 1]
        operations += 1
        if pattern[q] == text[i]:
            q += 1
        operations += 1
        if q == m:
            results.append((i + 1 - q, i))
            q = prefix_fun[q - 1]
    return results, operations

## Алгоритм Ахо-Карасика 

In [45]:
class Node:
    def __init__(self):
        self.next = {}
        self.fail = None
        self.is_word = False


class AhoCorasick:
    def __init__(self, patterns):
        self.root = Node()
        self.make_bor(patterns)
        self.set_links()

    def add_pattern(self, pattern):
        tmp = self.root
        for char in pattern:
            tmp = tmp.next.setdefault(char, Node())
        tmp.is_word = True

    def make_bor(self, patterns):
        if not isinstance(patterns, list):
            self.add_pattern(patterns)
        else:
            for pattern in patterns:
                self.add_pattern(pattern)

    def set_links(self):
        queue = [self.root]
        while queue:
            temp = queue.pop()
            p = None
            for key in temp.next.keys():
                if temp == self.root:
                    temp.next[key].fail = self.root
                else:
                    p = temp.fail
                    while p is not None:
                        if key in p.next:
                            temp.next[key].fail = p.next[key]
                            break
                        p = p.fail
                    if p is None:
                        temp.next[key].fail = self.root
                queue.append(temp.next[key])

    def search(self, text):
        results = set()
        start_index = 0

        for curr_position in range(len(text)):
            word = text[curr_position]
            end_index = curr_position
            p = self.root
            while word in p.next:
                if p == self.root:
                    start_index = curr_position
                p = p.next[word]
                if p.is_word:
                    if (start_index, end_index) not in results:
                        results.add((start_index, end_index))
                if p.next and end_index + 1 < len(text):
                    end_index += 1
                    word = text[end_index]
                else:
                    break
                while (word not in p.next) and (p != self.root):
                    p = p.fail
                    start_index += 1
                if p == self.root:
                    break
        return list(results)
    
def AC_matcher(text, pattern):
    operations = 0
    aho_corasick = AhoCorasick(pattern)
    return aho_corasick.search(text), operations

## Measurements of benchmark

In [46]:
def measure_function(fun, text, pattern, expected, iterations=1):
    total_time = 0
    for i in range(iterations):
        start = perf_counter()
        result = fun(text, pattern)
        if i == 0:
            if result[0] != expected:
                print("RESULT IS WRONG:", result, "EXPECTED:", expected)
            else:
                print('result:', result[0])
        end = perf_counter()
        total_time += (end - start)
    print('total time:', total_time / iterations)
    print('number of operations:', result[1], '\n')


def test(text, pattern):
    expected = naive_matcher(text, pattern)[0]
    
    print('\tНаивный алгоритм')
    measure_function(naive_matcher, text, pattern, expected)
    
    print("\tАлгоритм Рабина-Карпа" )
    measure_function(RK_matcher, text, pattern, expected)
    
    print('\tАлгоритм Бойера-Мура-Хорспула')
    measure_function(BMH_matcher, text, pattern, expected)
    
    print('\tАлгоритм Кнута-Морриса-Пратта')
    measure_function(KMP_matcher, text, pattern, expected)
    
    print('\tАлгоритм Ахо-Карасика')
    measure_function(AC_matcher, text, pattern, expected)
    
def start_testing():
    for bench_type in ['bad', 'good']:
        for i in range(1, 5):
            text = codecs.open(f"benchmarks/{bench_type}_t_{i}.txt", "r", "utf_8_sig")
            pattern = codecs.open(f"benchmarks/{bench_type}_w_{i}.txt", "r", "utf_8_sig")
            print(f'TEST FILE {bench_type} №{i}')
            test(text.read(), pattern.read())

In [47]:
start_testing()

TEST FILE bad №1
	Наивный алгоритм
result: [(8, 9)]
total time: 3.979999996772676e-05
number of operations: 18 

	Алгоритм Рабина-Карпа
result: [(8, 9)]
total time: 4.089999993084348e-05
number of operations: 11 

	Алгоритм Бойера-Мура-Хорспула
result: [(8, 9)]
total time: 0.0002318000001650944
number of operations: 11 

	Алгоритм Кнута-Морриса-Пратта
result: [(8, 9)]
total time: 3.3799999982875306e-05
number of operations: 28 

	Алгоритм Ахо-Карасика
result: [(8, 9)]
total time: 0.00034469999991415534
number of operations: 0 

TEST FILE bad №2
	Наивный алгоритм
result: [(90, 99)]
total time: 0.00019930000007661874
number of operations: 910 

	Алгоритм Рабина-Карпа
result: [(90, 99)]
total time: 7.460000006176415e-05
number of operations: 101 

	Алгоритм Бойера-Мура-Хорспула
result: [(90, 99)]
total time: 0.00023770000007061753
number of operations: 101 

	Алгоритм Кнута-Морриса-Пратта
result: [(90, 99)]
total time: 7.340000001931912e-05
number of operations: 290 

	Алгоритм Ахо-Караси